**This notebook is used for predicting whether the images are real or fake- VGG-16**

In [ ]:
#To clear the memory
import tensorflow as tf
tf.keras.backend.clear_session()

In [1]:
# Mounting collab to google collab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Unzipping the directory.
!unzip "/content/drive/MyDrive/deepfake.zip" -d "/content/dataset2"


Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset2/deepfake/validation/real/vehicles_05501.jpg  
  inflating: /content/dataset2/deepfake/validation/real/vehicles_05502.jpg  
  inflating: /content/dataset2/deepfake/validation/real/vehicles_05503.jpg  
  inflating: /content/dataset2/deepfake/validation/real/vehicles_05504.jpg  
  inflating: /content/dataset2/deepfake/validation/real/vehicles_05505.jpg  
  inflating: /content/dataset2/deepfake/validation/real/vehicles_05506.jpg  
  inflating: /content/dataset2/deepfake/validation/real/vehicles_05507.jpg  
  inflating: /content/dataset2/deepfake/validation/real/vehicles_05508.jpg  
  inflating: /content/dataset2/deepfake/validation/real/vehicles_05509.jpg  
  inflating: /content/dataset2/deepfake/validation/real/vehicles_05510.jpg  
  inflating: /content/dataset2/deepfake/validation/real/vehicles_05511.jpg  
  inflating: /content/dataset2/deepfake/validation/real/vehicles_05512.jpg  
  inflating: /content/dat

In [29]:
####VGG -16 ###############
import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow import keras



print(f"Num GPUs Available: {len(tf.config.experimental.list_physical_devices('GPU'))}")

strategy = tf.distribute.MirroredStrategy()

print(f"Number of devices: {strategy.num_replicas_in_sync}")

with strategy.scope():
    base_model = tf.keras.applications.VGG16(
        input_shape=(224, 224, 3),
        weights="imagenet",
        include_top=False
    )

    base_model.trainable = False  # Freezing the base model layers

    flatten_in = tf.keras.layers.Flatten()(base_model.output)

    fc1 = tf.keras.layers.Dense(
        2048,
        activation="relu",
        # kernel_regularizer=regularizers.l2(0.01)
    )(flatten_in)

    fc2 = tf.keras.layers.Dense(
        2048,
        activation="relu",
        # kernel_regularizer=regularizers.l2(0.01)
    )(fc1)

    prediction = tf.keras.layers.Dense(
        units=2,
        activation="softmax",
        # kernel_regularizer=regularizers.l2(0.01)
    )(fc2)

    model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss="categorical_crossentropy",
        metrics=['accuracy']
    )

# Model summary
model.summary()




Num GPUs Available: 1
Number of devices: 1


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 2048)           │    51,382,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 2048)           │     4,196,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 2)              │         4,098 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 70,297,410 (268.16 MB)

 Trainable params: 55,582,722 (212.03 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [37]:
#Data splitting using flow_from_directory
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import preprocess_input

datagenerator_kwargs = dict(
    preprocessing_function=preprocess_input,
)

dataflow_kwargs = dict(
    target_size=(224, 224),  # Resize all images to 224x224
    batch_size=256,
    interpolation="bilinear"  # Image resizing interpolation method
)

valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
    **datagenerator_kwargs
)

valid_generator = valid_datagenerator.flow_from_directory(
    directory='/content/dataset2/deepfake/validation',
    shuffle=False,
    **dataflow_kwargs
)

# Training data generator (With Augmentations). It generates data on fly
train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=30,
    vertical_flip=True,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.3,
    brightness_range=[0.3, 1.7],
    **datagenerator_kwargs
)

train_generator = train_datagenerator.flow_from_directory(
    directory='/content/dataset2/deepfake/train',  # Training dataset
    shuffle=True,
    **dataflow_kwargs
)


Found 60000 images belonging to 2 classes.
Found 167998 images belonging to 2 classes.


In [38]:
#Fake -0
# Real -1
import json
import os

# Get class indices from the train_generator
class_indices = train_generator.class_indices

# Reverse to map indices to class names
class_names = {v: k for k, v in class_indices.items()}

# Save to a JSON file
with open('class_indices.json', 'w') as f:
    json.dump(class_indices, f)

print("Class indices saved to class_indices.json")

Class indices saved to class_indices.json


In [39]:
# Total number of data
train_class_counts = train_generator.class_indices
print(f"Training class indices: {train_class_counts}")

train_class_counts = {class_name: 0 for class_name in train_generator.class_indices.keys()}
for class_name, class_index in train_generator.class_indices.items():
    train_class_counts[class_name] = sum(train_generator.classes == class_index)

print(f"Training data class distribution: {train_class_counts}")

valid_class_counts = valid_generator.class_indices
print(f"Validation class indices: {valid_class_counts}")

valid_class_counts = {class_name: 0 for class_name in valid_generator.class_indices.keys()}
for class_name, class_index in valid_generator.class_indices.items():
    valid_class_counts[class_name] = sum(valid_generator.classes == class_index)

print(f"Validation data class distribution: {valid_class_counts}")


Training class indices: {'fake': 0, 'real': 1}
Training data class distribution: {'fake': np.int64(83999), 'real': np.int64(83999)}
Validation class indices: {'fake': 0, 'real': 1}
Validation data class distribution: {'fake': np.int64(30000), 'real': np.int64(30000)}


In [40]:
# Saving the callbacks i.e. best model
import os
from tensorflow.keras.callbacks import ModelCheckpoint

save_dir = '/content/drive/MyDrive/deepvsfake_models'
os.makedirs(save_dir, exist_ok=True)

checkpoint = ModelCheckpoint(
    os.path.join(save_dir, "best_model_epoch_deepvsfake{epoch:02d}.h5"),
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)


In [41]:
#Early stopping 6
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor="val_accuracy", patience=6
)

In [42]:
##Interrupted the model because of time constraint
model.fit(
              train_generator,
              epochs=30,
              validation_data=valid_generator,
              callbacks=[checkpoint,early_stop]
              # class_weight=class_weights

          )

Epoch 1/30
657/657 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.6415 - loss: 7.5990
Epoch 1: val_accuracy improved from -inf to 0.71297, saving model to /content/drive/MyDrive/deepvsfake_models/best_model_epoch_deepvsfake01.h5


657/657 ━━━━━━━━━━━━━━━━━━━━ 3166s 5s/step - accuracy: 0.6416 - loss: 7.5902 - val_accuracy: 0.7130 - val_loss: 0.5355
Epoch 2/30
657/657 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.7012 - loss: 0.5538
Epoch 2: val_accuracy improved from 0.71297 to 0.71553, saving model to /content/drive/MyDrive/deepvsfake_models/best_model_epoch_deepvsfake02.h5


657/657 ━━━━━━━━━━━━━━━━━━━━ 3136s 5s/step - accuracy: 0.7012 - loss: 0.5538 - val_accuracy: 0.7155 - val_loss: 0.5407
Epoch 3/30
657/657 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.7048 - loss: 0.5524
Epoch 3: val_accuracy improved from 0.71553 to 0.71760, saving model to /content/drive/MyDrive/deepvsfake_models/best_model_epoch_deepvsfake03.h5


657/657 ━━━━━━━━━━━━━━━━━━━━ 3115s 5s/step - accuracy: 0.7048 - loss: 0.5523 - val_accuracy: 0.7176 - val_loss: 0.5386
Epoch 4/30
  7/657 ━━━━━━━━━━━━━━━━━━━━ 49:00 5s/step - accuracy: 0.7147 - loss: 0.5462

KeyboardInterrupt: 

In [44]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input
import tensorflow as tf
import pandas as pd
import numpy as np
import os

# Set test directory
test_dir = '/content/dataset2/deepfake/test'

# Get test image paths
test_images = [os.path.join(test_dir, f) for f in os.listdir(test_dir)
              if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
df_test = pd.DataFrame({'filename': test_images})

# Create ImageDataGenerator
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=df_test,
    x_col='filename',
    y_col=None,  # No labels
    target_size=(224, 224),
    batch_size=1,
    shuffle=False,
    class_mode=None
)

# Loading the model
model = tf.keras.models.load_model('/content/drive/MyDrive/deepvsfake_models/best_model_epoch_deepvsfake03.h5')

# Get predictions
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Binary classification mapping (0=fake, 1=real)
binary_labels = [0 if cls == 0 else 1 for cls in predicted_classes]  # Assuming fake=0, real=1

results = pd.DataFrame({
    'image': [os.path.basename(f) for f in test_images],
    'label': binary_labels  # 0 for fake, 1 for real
})

results.to_csv('predictions2.csv', index=False)
print(f"Predictions saved for {len(results)} images (0=fake, 1=real)")

Found 12002 validated image filenames.


12002/12002 ━━━━━━━━━━━━━━━━━━━━ 125s 10ms/step
Predictions saved for 12002 images (0=fake, 1=real)
